# Second Quantized Hamiltonian

In [1]:
import numpy as np

from classiq import construct_chemistry_model, execute, show, synthesize
from classiq.applications.chemistry import (
    ChemistryExecutionParameters,
    FermionicOperator,
    HamiltonianProblem,
    HEAParameters,
    SummedFermionicOperator,
)
from classiq.execution import OptimizerType

In [2]:
hamiltonian = SummedFermionicOperator(
    op_list=[
        (FermionicOperator(op_list=[("+", 0), ("-", 0)]), 0.2),
        (FermionicOperator(op_list=[("-", 1), ("-", 1)]), 0.3),
        (FermionicOperator(op_list=[("-", 2), ("-", 2)]), 0.4),
        (FermionicOperator(op_list=[("-", 3), ("-", 3)]), 0.5),
        (FermionicOperator(op_list=[("+", 0), ("+", 1), ("-", 1), ("-", 0)]), -0.1),
        (FermionicOperator(op_list=[("+", 2), ("+", 3), ("-", 2), ("-", 3)]), -0.3),
    ]
)
ham_problem = HamiltonianProblem(hamiltonian=hamiltonian, num_particles=[1, 1])

print(ham_problem.hamiltonian)

0.2 * a⁺₀a₀ + 
0.3 * a₁a₁ + 
0.4 * a₂a₂ + 
0.5 * a₃a₃ + 
-0.1 * a⁺₀a⁺₁a₁a₀ + 
-0.3 * a⁺₂a⁺₃a₂a₃


In [3]:
hwea_params = HEAParameters(
    num_qubits=4,
    connectivity_map=[(0, 1), (1, 2), (2, 3)],
    reps=3,
    one_qubit_gates=["x", "ry"],
    two_qubit_gates=["cx"],
)

qmod = construct_chemistry_model(
    chemistry_problem=ham_problem,
    use_hartree_fock=False,
    ansatz_parameters=hwea_params,
    execution_parameters=ChemistryExecutionParameters(
        optimizer=OptimizerType.COBYLA,
        max_iteration=100,
        initial_point=None,
    ),
)

In [4]:
from classiq import write_qmod

write_qmod(qmod, name="second_quantized_hamiltonian")

In [5]:
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/2806f7a1-c17c-4a05-9abb-d19786e25884?version=0.41.0.dev39%2B79c8fd0855


In [6]:
results = execute(qprog).result()
chemistry_result = results[0].value
chemistry_result.energy

0.0006000000000000068